# Data Frame Federated Statistics 

In this example, we will show how to generate federated statistics for data that can be represented as Pandas Data Frame.

## Set Up NVFLARE

Follow [Getting Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to set up a virtual environment and install NVFLARE.


## Install requirements
First, install the required packages:

In [ ]:
%pip install -r df_stats/requirements.txt


## Prepare data

In this example, we are using UCI (University of California, Irvine) [adult dataset](https://archive.ics.uci.edu/dataset/2/adult)
The original dataset has already contains "training" and "test" datasets. Here we simply assume that "training" and test data sets are belong to different clients.
so we assigned the training data and test data into two clients.
 
Now we use data utility to download UCI datasets to separate client package directory to /tmp/nvflare/data/ directory.
Please note that the UCI's website may experience occasional downtime.



In [ ]:
from df_stats.utils.prepare_data import prepare_data

prepare_data(data_root_dir = "/tmp/nvflare/df_stats/data")

#### Let's take a look at the data

In [ ]:
import pandas as pd
data_path ="/tmp/nvflare/df_stats/data/site-1/data.csv"
data_features = [
            "Age",
            "Workclass",
            "fnlwgt",
            "Education",
            "Education-Num",
            "Marital Status",
            "Occupation",
            "Relationship",
            "Race",
            "Sex",
            "Capital Gain",
            "Capital Loss",
            "Hours per week",
            "Country",
            "Target",
        ]

        # the original dataset has no header,
        # we will use the adult.train dataset for site-1, the adult.test dataset for site-2
        # the adult.test dataset has incorrect formatted row at 1st line, we will skip it.
skip_rows = {
            "site-1": [],
            "site-2": [0],
        }

df= pd.read_csv(data_path, names=data_features, sep=r"\s*,\s*", skiprows=skip_rows, engine="python", na_values="?")
df


> Note **We will only calculate the statistics of numerical features, categorical features will be skipped**

## Run job in FL Simulator

**Run Job using Simulator API**


In [7]:
! nvflare simulator df_stats/jobs/df_stats -w /tmp/nvflare/df/workdir -n 2 -t 2

2024-12-28 21:02:38,885 - SimulatorRunner - INFO - Create the Simulator Server.
2024-12-28 21:02:38,886 - CoreCell - INFO - server: creating listener on tcp://0:56065
2024-12-28 21:02:38,997 - CoreCell - INFO - server: created backbone external listener for tcp://0:56065
2024-12-28 21:02:38,997 - ConnectorManager - INFO - 36336: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2024-12-28 21:02:38,998 - conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:17913] is starting
2024-12-28 21:02:39,503 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:17913
2024-12-28 21:02:39,504 - conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:56065] is starting
2024-12-28 21:02:39,506 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Request
2024-12-28 21:02:39,506 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Abort
2024-12-28 21:02:39,506 - SimulatorServerEngine - INFO - initialize_comm c

Or python code
```
from nvflare.private.fed.app.simulator.simulator_runner import SimulatorRunner
runner = SimulatorRunner(job_folder="df_stats/jobs/df_stats", workspace="/tmp/nvflare/df/workdir", n_clients = 2, threads=2)
runner.run()

```

**Run Job using Simulator CLI**

From a **terminal** one can also the following equivalent CLI

```
nvflare simulator df_stats/jobs/df_stats -w /tmp/nvflare/df/workdir -n 2 -t 2

```

assuming the nvflare is installed from a **terminal**. doing pip install from the notebook cell directory with bash command (! or %%bash) may or may not work depending on which python runtime kernel selected. Also %pip install or %pip install from notebook cell doesn't register the console_scripts in the PATH.   


## Examine the result




The results are stored in workspace "/tmp/nvflare/df/workdir/"
```
/tmp/nvflare/df/workdir/server/simulate_job/statistics/adults_stats.json
```

In [8]:
cat /tmp/nvflare/df/workdir/server/simulate_job/statistics/adults_stats.json

{"Age": {"count": {"site-2": {"train": 13025, "test": 3256}, "site-1": {"train": 26049, "test": 6512}, "Global": {"train": 39074, "test": 9768}}, "mean": {"site-2": {"train": 38.75, "test": 38.84}, "site-1": {"train": 38.55, "test": 38.69}, "Global": {"train": 38.62, "test": 38.74}}, "sum": {"site-2": {"train": 504707, "test": 126466}, "site-1": {"train": 1004296, "test": 251961}, "Global": {"train": 1509003, "test": 378427}}, "percentile": {"site-2": {"train": {"25": 27.68, "50": 37.17, "75": 47.84}, "test": {"25": 27.8, "50": 37.01, "75": 48.12}}, "site-1": {"train": {"25": 27.59, "50": 36.96, "75": 47.62}, "test": {"25": 27.71, "50": 37.31, "75": 47.69}}, "Global": {"train": {"25": 27.62, "50": 37.03, "75": 47.7}, "test": {"25": 27.74, "50": 37.22, "75": 47.84}}}, "stddev": {"site-2": {"train": 13.82, "test": 13.95}, "site-1": {"train": 13.66, "test": 13.56}, "Global": {"train": 13.71, "test": 13.69}}, "histogram": {"site-2": {"train": [[0.0, 12.0, 0], [12.0, 24.0, 1992], [24.0, 36.

## Visualization
We can visualize the results easly via the visualizaiton notebook. Before we do that, we need to copy the data to the notebook directory 


In [9]:
! cp /tmp/nvflare/df/workdir/server/simulate_job/statistics/adults_stats.json df_stats/demo/.

now we can visualize via the [visualization notebook](df_stats/demo/visualization.ipynb)

We are not quite done yet. What if you prefer to use python API instead CLI to run jobs. Lets do that in this section

## We are done !
Congratulations, you just completed the federated stats calulation with data represented by data frame
